<a href="https://colab.research.google.com/github/ramezanipourn/Fashion_Mnist_Classification/blob/main/Assignment_s1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
from tqdm import tqdm
from torch.utils.data import DataLoader
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
transform = torchvision.transforms.Compose([
                                    torchvision.transforms.ToTensor(),
                                    # torchvision.transforms.RandomHorizontalFlip(),
                                    torchvision.transforms.Normalize(mean=0.5,std=0.5)
])

In [4]:
from torchvision.datasets import FashionMNIST
train_data = FashionMNIST('dataset',train=True,download=True,transform = transform)
test_data = FashionMNIST('dataset',train=False,download=True,transform = transform)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting dataset/FashionMNIST/raw/train-images-idx3-ubyte.gz to dataset/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting dataset/FashionMNIST/raw/train-labels-idx1-ubyte.gz to dataset/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting dataset/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to dataset/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting dataset/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/FashionMNIST/raw



In [18]:
class MyModel(torch.nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()
    self.name = 'MyModel'
    self.fc1 = torch.nn.Linear(784,256)
    self.fc2 = torch.nn.Linear(256,128)
    self.fc3 = torch.nn.Linear(128,10)

  def forward(self,x):
    # input shape = 32 * 28 * 28
    x = x.reshape((x.shape[0],784))
    x = self.fc1(x)
    x = torch.relu(x)
    x = torch.dropout(x,0.2,train=True)
    x = self.fc2(x)
    x = torch.relu(x)
    x = torch.dropout(x,0.2,train=True)
    x = self.fc3(x)
    x = torch.softmax(x,dim=1)
    return x


In [6]:
# !pip install ipykernel
# !python -m ipykernel install


In [19]:
from argparse import ArgumentParser
parser = ArgumentParser()

parser.add_argument("--root_dir", default='drive/MyDrive/')
parser.add_argument("--ckpt_dir", default=os.path.join('drive/MyDrive/', 'checkpoints'))
parser.add_argument("--lr", help="learning rate", type=float, default=1e-3)
parser.add_argument("--batch_size", help="batch size", type=int, default=32)
parser.add_argument("--epochs", help="number of epochs", type=int, default=50)
parser.add_argument("--numworker", help="number of threads", type=int, default=2)
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
args = parser.parse_args()

In [20]:
lr = args.lr
epochs =args.epochs
bachsize = args.batch_size
numworker = args.numworker
root_dir = args.root_dir
ckpt_dir = args.ckpt_dir

In [21]:
train_loader = DataLoader(train_data,batch_size=bachsize, shuffle=True,num_workers=numworker)
test_loader = DataLoader(train_data,batch_size=bachsize, shuffle=True,num_workers=numworker)

In [22]:
model = MyModel()

In [25]:
class TrainTest:
  def __init__(self,model):

    self.device =  torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    self.model = model.to(self.device)
    print(next(model.parameters()).device)
    os.makedirs(os.path.join(ckpt_dir, self.model.name), exist_ok=True)

    '''criterion'''
    self.criterion = torch.nn.CrossEntropyLoss().to(self.device)
               
    '''optimizer'''
    self.optimizer = torch.optim.Adam(self.model.parameters(),lr= lr)
    '''scheduler'''
    self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, mode= 'max',factor=0.5,patience=5)
    
  # def calc_acc(preds, labels):
  #       _, pred_max = torch.max(preds, 1)
  #       acc = torch.sum(pred_max == labels.data, dtype=torch.float64) / len(preds)
  #       return acc

  def train_(self):
    total_train_loss = 0.0
    for epoch in range(epochs):
      self.model.train(True)
      train_loss = 0.0
      acc = 0.0
      for image , lable in tqdm(train_loader):
        
        image = image.to(self.device)
        lable = lable.to(self.device)
        predict = self.model(image)
        
        loss = self.criterion( predict,lable)
        loss.backward()
        self.optimizer.zero_grad()
        self.optimizer.step()
        train_loss += loss
        # acc += self.calc_acc(predict, lable)
        _, pred_max = torch.max(predict, 1)
        acc += torch.sum(pred_max == lable.data, dtype=torch.float64) / len(predict)

      epoch_loss = train_loss / len(train_loader) 
      epoch_acc =acc / len(train_loader)
      self.scheduler.step(epoch_loss)
      print('epoch:{} | train_loss: {:.4f}, train_accuracy :{:.2f}'.format(epoch+1, epoch_loss,epoch_acc))

      if epoch % 10== 0:
                torch.save(self.model.state_dict(),
                           os.path.join(ckpt_dir,self.model.name, f'ckpt_{epoch}.pth'))
      if epoch % 5== 0:
        self.test()

  def test(self):
      for image , lable in tqdm(test_loader):
        self.model.eval()
        test_loss = 0.0
        acc = 0.0
        image = image.to(self.device)
        lable = lable.to(self.device)
        predict = self.model(image)
        loss = self.criterion(predict,lable)
        
        test_loss += loss
        # acc += self.calc_acc(predict, lable)
        _, pred_max = torch.max(predict, 1)
        acc += torch.sum(pred_max == lable.data, dtype=torch.float64) / len(predict)


      epoch_loss = test_loss / len(test_loader)
      epoch_acc =acc / len(test_loader)
      print(' test_loss: {:.4f}, test_accuracy:{:.2f}'.format( epoch_loss,epoch_acc))
        




In [26]:
train_test = TrainTest(model)
train_test.train_()

cuda:0


100%|██████████| 1875/1875 [00:19<00:00, 96.46it/s]


epoch:1 | train_loss: 2.3029, train_accuracy :0.13


100%|██████████| 1875/1875 [00:15<00:00, 117.76it/s]


 test_loss: 0.0012, test_accuracy:0.00


...

epoch:48 | train_loss: 2.3029, train_accuracy :0.13


100%|██████████| 1875/1875 [00:19<00:00, 95.94it/s] 


epoch:49 | train_loss: 2.3029, train_accuracy :0.13


100%|██████████| 1875/1875 [00:19<00:00, 96.45it/s]

epoch:50 | train_loss: 2.3029, train_accuracy :0.13
